# Parker Solar Probe <a class="anchor" id="psp"></a>

In [58]:
# to do:
# polarity_rtn, polarity_panel, polarity_colorwheel

## Importing and general settings
Only needed to run once

In [59]:
import astropy.units as u
import datetime as dt
# import math
import numpy as np
import os
import pandas as pd
import re
import sunpy
import warnings

from astropy.constants import e, k_B, m_p
from astropy.table import QTable
from matplotlib import cm
from matplotlib import pyplot as plt
# from matplotlib.colors import LogNorm
from matplotlib.colors import Normalize
import matplotlib.dates as mdates
from mpl_toolkits.axes_grid1.inset_locator import inset_axes
from seppy.loader.psp import calc_av_en_flux_PSP_EPILO, calc_av_en_flux_PSP_EPIHI, psp_isois_load
from seppy.tools import resample_df
from stixdcpy.quicklook import LightCurves # https://github.com/i4Ds/stixdcpy
from sunpy.coordinates import frames, get_horizons_coord
from sunpy.net import attrs as a
from sunpy.net import Fido
from sunpy.timeseries import TimeSeries

# from my_func_py3 import polarity_rtn
from polarity_plotting import polarity_rtn, polarity_panel, polarity_colorwheel

# disable unused speasy data provider before importing to speed it up
os.environ['SPEASY_CORE_DISABLED_PROVIDERS'] = "sscweb,archive,csa"
import speasy as spz

# omit Pandas' PerformanceWarning
warnings.simplefilter(action='ignore', category=pd.errors.PerformanceWarning)

plt.rcParams['axes.linewidth'] = 1.5
plt.rcParams['font.size'] = 12
plt.rcParams['xtick.labelsize'] = 15
plt.rcParams['ytick.labelsize'] = 15
plt.rc('axes', titlesize=20)  # fontsize of the axes title
plt.rc('axes', labelsize=20)  # fontsize of the x and y labels
plt.rcParams['agg.path.chunksize'] = 20000

## Define general options

In [60]:
#Timestamp position
pos_timestamp = None

#Path to save data
#file_path = 'C:\Users\osant\Desktop\solar_physics_projects\data'

year = 2023
startdate = dt.datetime(year, 3, 12, 12, 00)
enddate = dt.datetime(year, 3, 15, 12, 00)
# startdate = dt.datetime(year, 10, 3, 0, 00)
# enddate = dt.datetime(year, 10, 6)

plot_stix = True
plot_epilo_e = True
plot_epihi_e = False
plot_epilo_p = False
plot_epihi_p = True
# plot_psp_pixel = True
plot_epihi_p_combined_pixels = True

plot_fields = True

plot_pad = False
plot_mag_angles = True 
plot_mag = True
plot_Vsw = True
plot_N = True
plot_T = False
plot_p_dyn = False
plot_polarity = True 


legends_inside = False

resample = '5min'
resample_mag = '1min'

psp_het_viewing = 'A'  # 'A' or 'B'?

epilo_channel = 'F'
epilo_viewing = '3'
epilo_ic_channel = 'T'
epilo_ic_viewing = '3'

stix_ltc = False  # correct SolO/STIX data for light travel time

Don't change the following cell, just run:

In [61]:
if plot_epilo_e or plot_epihi_e:
    plot_electrons = True
else:
    plot_electrons = False

if plot_epilo_p or plot_epihi_p:
    plot_protons = True
else:
    plot_protons = False

panels = 1*plot_stix + 1*plot_electrons + 1*plot_protons + 1*plot_pad + 2*plot_mag_angles + 1*plot_mag + 1*plot_Vsw + 1*plot_N + 1*plot_T + 1*plot_p_dyn

panel_ratios = list(np.zeros(panels)+1)
if plot_electrons and plot_protons:
    panel_ratios[0+1*plot_stix] = 2
    panel_ratios[1+1*plot_stix] = 2
if plot_electrons or plot_protons:    
    panel_ratios[0+1*plot_stix] = 2

## Data loading

### Load SolO/STIX data

In [62]:
if plot_stix:
    if enddate-startdate > dt.timedelta(days=7):
        print('WARNING: STIX loading for more than 7 days not supported at the moment!')
        print('')
    lc = LightCurves.from_sdc(start_utc=startdate, end_utc=enddate, ltc=stix_ltc)
    df_stix_orig = lc.to_pandas()

### Load PSP data

In [63]:
# NOT IN USE HERE: Manual loading of private data

# psp_man_path = 'psp_private_data/'

# df_psp_pixel_org = pd.DataFrame()
# for key in ['L2Ap', 'L3Ap', 'L4Ap', 'H2Ap', 'H3Ap', 'H4Ap']:
#     tdf = pd.read_csv(psp_man_path+f'{key}_flux60_full.csv', skiprows=25, date_format='%Y-%jT%H:%M:%S.%f', index_col=0, names=[f'{key}_Flux'])
#     df_psp_pixel_org = pd.concat([df_psp_pixel_org, tdf], axis=1)

# energies_psp_pixel = {}
# energies_psp_pixel['L2Ap'] = '5.22 - 12.20 MeV'
# energies_psp_pixel['L3Ap'] = '11.49 - 20.76 MeV'
# energies_psp_pixel['H2Ap'] = '12.95 - 18.08 MeV'
# energies_psp_pixel['L4Ap'] = '17.63 - 26.35 MeV'
# energies_psp_pixel['H3Ap'] = '23.65 - 27.53 MeV'
# energies_psp_pixel['H4Ap'] = '31.04 - 36.19 MeV'

In the following, loading of public level-2 data

In [64]:
df_psp_het_org, psp_het_energies = psp_isois_load('PSP_ISOIS-EPIHI_L2-HET-RATES60', startdate, enddate, path=None, resample=None)

2025-02-18 11:12:06 - sunpy - WARNING: SunpyUserWarning: The variable "HET_A_R17_SECT_HCI" has been skipped because it has more than 2 dimensions, which is unsupported.
2025-02-18 11:12:06 - sunpy - WARNING: SunpyUserWarning: The variable "HET_A_R17_SECT_RTN" has been skipped because it has more than 2 dimensions, which is unsupported.
2025-02-18 11:12:06 - sunpy - WARNING: SunpyUserWarning: The variable "HET_B_R17_SECT_HCI" has been skipped because it has more than 2 dimensions, which is unsupported.
2025-02-18 11:12:06 - sunpy - WARNING: SunpyUserWarning: The variable "HET_B_R17_SECT_RTN" has been skipped because it has more than 2 dimensions, which is unsupported.
2025-02-18 11:12:07 - sunpy - WARNING: SunpyUserWarning: The variable "HET_A_R17_SECT_HCI" has been skipped because it has more than 2 dimensions, which is unsupported.
2025-02-18 11:12:07 - sunpy - WARNING: SunpyUserWarning: The variable "HET_A_R17_SECT_RTN" has been skipped because it has more than 2 dimensions, which is

In [65]:
df_psp_epilo_org, psp_epilo_energies_org = psp_isois_load('PSP_ISOIS-EPILO_L2-PE', startdate, enddate, path=None, resample=None, epilo_channel=epilo_channel, epilo_threshold=None)

In [66]:
df_psp_epilo_ic_org, psp_epilo_ic_energies_org = psp_isois_load('PSP_ISOIS-EPILO_L2-IC', startdate, enddate, path=None, resample=None, epilo_channel=epilo_ic_channel, epilo_threshold=None)

In [67]:
# for i in range(47):
#     print(f'H_Chan{epilo_ic_channel}_Energy_E{i}_P{epilo_ic_viewing}', psp_epilo_ic_energies_org[f'H_Chan{epilo_ic_channel}_Energy'][f'H_Chan{epilo_ic_channel}_Energy_E{i}_P{epilo_ic_viewing}'])

In [68]:
# SPC
df_psp_spc_np_tot = spz.get_data(spz.inventories.data_tree.amda.Parameters.PSP.SWEAP_SPC.psp_spc_fit.psp_spc_np_tot, 
                            startdate, enddate, output_format="CDF_ISTP").replace_fillval_by_nan().to_dataframe()
df_psp_spc_vp_tot_nrm = spz.get_data(spz.inventories.data_tree.amda.Parameters.PSP.SWEAP_SPC.psp_spc_fit.psp_spc_vp_tot_nrm, 
                            startdate, enddate, output_format="CDF_ISTP").replace_fillval_by_nan().to_dataframe()
df_psp_spc_vp_tot_rtn = spz.get_data(spz.inventories.data_tree.amda.Parameters.PSP.SWEAP_SPC.psp_spc_fit.psp_spc_vp_tot, 
                            startdate, enddate, output_format="CDF_ISTP").replace_fillval_by_nan().to_dataframe()
df_psp_spc_wp_tot = spz.get_data(spz.inventories.data_tree.amda.Parameters.PSP.SWEAP_SPC.psp_spc_fit.psp_spc_wp_tot, 
                            startdate, enddate, output_format="CDF_ISTP").replace_fillval_by_nan().to_dataframe()
df_psp_spc_GF = spz.get_data(spz.inventories.data_tree.amda.Parameters.PSP.SWEAP_SPC.psp_spc_flag.psp_spc_gf, 
                            startdate, enddate, output_format="CDF_ISTP").replace_fillval_by_nan().to_dataframe()
df_psp_spc = pd.concat([df_psp_spc_np_tot, df_psp_spc_vp_tot_nrm, df_psp_spc_vp_tot_rtn, df_psp_spc_wp_tot, df_psp_spc_GF], axis=1)

# SPAN-i
df_psp_spani_np = spz.get_data(spz.inventories.data_tree.cda.ParkerSolarProbe.PSPSWEAPSPAN.PSP_SWP_SPI_SF00_L3_MOM.DENS, 
                            startdate, enddate).replace_fillval_by_nan().to_dataframe()
df_psp_spani_vp_rtn_sun = spz.get_data(spz.inventories.data_tree.cda.ParkerSolarProbe.PSPSWEAPSPAN.PSP_SWP_SPI_SF00_L3_MOM.VEL_RTN_SUN, 
                            startdate, enddate).replace_fillval_by_nan().to_dataframe()
df_psp_spani_T = spz.get_data(spz.inventories.data_tree.cda.ParkerSolarProbe.PSPSWEAPSPAN.PSP_SWP_SPI_SF00_L3_MOM.TEMP, 
                            startdate, enddate).replace_fillval_by_nan().to_dataframe()
df_psp_spani_QF = spz.get_data(spz.inventories.data_tree.cda.ParkerSolarProbe.PSPSWEAPSPAN.PSP_SWP_SPI_SF00_L3_MOM.QUALITY_FLAG, 
                            startdate, enddate).replace_fillval_by_nan().to_dataframe()
df_psp_spani = pd.concat([df_psp_spani_np, df_psp_spani_vp_rtn_sun, df_psp_spani_T, df_psp_spani_QF], axis=1)

# MAG
df_psp_mag_rtn = spz.get_data(spz.inventories.data_tree.amda.Parameters.PSP.FIELDS_MAG.psp_mag_1min.psp_b_1min, 
                            startdate, enddate, output_format="CDF_ISTP").replace_fillval_by_nan().to_dataframe()
df_psp_mag_phi = spz.get_data(spz.inventories.data_tree.amda.Parameters.PSP.FIELDS_MAG.psp_mag_1min.psp_b_1min_phi, 
                           startdate, enddate, output_format="CDF_ISTP").replace_fillval_by_nan().to_dataframe()
df_psp_mag_theta = spz.get_data(spz.inventories.data_tree.amda.Parameters.PSP.FIELDS_MAG.psp_mag_1min.psp_b_1min_theta, 
                            startdate, enddate, output_format="CDF_ISTP").replace_fillval_by_nan().to_dataframe()
df_psp_mag_tot = spz.get_data(spz.inventories.data_tree.amda.Parameters.PSP.FIELDS_MAG.psp_mag_1min.psp_b_1min_tot, 
                            startdate, enddate, output_format="CDF_ISTP").replace_fillval_by_nan().to_dataframe()
df_psp_mag = pd.concat([df_psp_mag_rtn, df_psp_mag_phi, df_psp_mag_theta, df_psp_mag_tot], axis=1)
df_psp_mag['phi_mod'] = ((df_psp_mag['phi'].values - 180) % 360) - 180 

c:\Users\osant\miniforge3\envs\sp2\Lib\site-packages\speasy\core\data_containers.py:17: UserWarning: no explicit representation of timezones available for np.datetime64
  return np.searchsorted(time, np.datetime64(key, 'ns'), side='left')
c:\Users\osant\miniforge3\envs\sp2\Lib\site-packages\speasy\core\data_containers.py:17: UserWarning: no explicit representation of timezones available for np.datetime64
  return np.searchsorted(time, np.datetime64(key, 'ns'), side='left')
c:\Users\osant\miniforge3\envs\sp2\Lib\site-packages\speasy\core\data_containers.py:17: UserWarning: no explicit representation of timezones available for np.datetime64
  return np.searchsorted(time, np.datetime64(key, 'ns'), side='left')


In [69]:
def mag_angles(B,Br,Bt,Bn):
    theta = np.arccos(Bn/B)
    alpha = 90-(180/np.pi*theta)

    r = np.sqrt(Br**2 + Bt**2 + Bn**2)
    phi = np.arccos(Br/np.sqrt(Br**2 + Bt**2))*180/np.pi

    sel = np.where(Bt < 0)
    count = len(sel[0])
    if count > 0:
        phi[sel] = 2*np.pi - phi[sel]
    sel = np.where(r <= 0)
    count = len(sel[0])
    if count > 0:
        phi[sel] = 0

    return alpha, phi

if plot_mag_angles:
    theta, phi = mag_angles(df_psp_mag['|b|'].values, df_psp_mag['br'].values, df_psp_mag['bt'].values, df_psp_mag['bn'].values)
    df_psp_mag['theta2'] = theta
    df_psp_mag['phi2'] = phi

#### Convert to Astropy QTable
Read units into dictionary

In [ ]:
df_psp_spc_units = {}
df_psp_spc_units['np_tot'] = u.Unit(spz.inventories.data_tree.amda.Parameters.PSP.SWEAP_SPC.psp_spc_fit.psp_spc_np_tot.units)
df_psp_spc_units['|vp_tot|'] = u.Unit(spz.inventories.data_tree.amda.Parameters.PSP.SWEAP_SPC.psp_spc_fit.psp_spc_vp_tot_nrm.units)
for k in ['vp_totr', 'vp_tott', 'vp_totn']:
    df_psp_spc_units[k] = u.Unit(spz.inventories.data_tree.amda.Parameters.PSP.SWEAP_SPC.psp_spc_fit.psp_spc_vp_tot.units)
df_psp_spc_units['wp_tot'] = u.Unit(spz.inventories.data_tree.amda.Parameters.PSP.SWEAP_SPC.psp_spc_fit.psp_spc_wp_tot.units)

df_psp_spani_units = {}
df_psp_spani_units['Density'] = u.Unit(spz.inventories.data_tree.cda.ParkerSolarProbe.PSPSWEAPSPAN.PSP_SWP_SPI_SF00_L3_MOM.DENS.UNITS)
for k in ['Vx RTN', 'Vy RTN', 'Vz RTN']:
    df_psp_spani_units[k] = u.Unit(spz.inventories.data_tree.cda.ParkerSolarProbe.PSPSWEAPSPAN.PSP_SWP_SPI_SF00_L3_MOM.VEL_RTN_SUN.UNITS)
df_psp_spani_units['Temperature'] = u.Unit(spz.inventories.data_tree.cda.ParkerSolarProbe.PSPSWEAPSPAN.PSP_SWP_SPI_SF00_L3_MOM.TEMP.UNITS)

In [ ]:
qt_psp_spc = QTable.from_pandas(df_psp_spc, index=True, units=df_psp_spc_units)
# qt_psp_spc = QTable(qt_psp_spc, masked=True)

In [ ]:
qt_psp_spani = QTable.from_pandas(df_psp_spani, index=True, units=df_psp_spani_units)

In [ ]:
qt_psp_spc['T'] = (1/2*m_p/k_B*(qt_psp_spc['wp_tot'])**2).si
qt_psp_spc['p_dyn'] = (m_p*qt_psp_spc['np_tot']*(qt_psp_spc['|vp_tot|'])**2).to(u.nPa)

In [ ]:
qt_psp_spani['V_tot_rtn'] = np.sqrt(qt_psp_spani['Vx RTN']**2+qt_psp_spani['Vy RTN']**2+qt_psp_spani['Vz RTN']**2)
qt_psp_spani['T_K'] = (qt_psp_spani['Temperature']/k_B).si
qt_psp_spani['p_dyn'] = (m_p*qt_psp_spani['Density']*(qt_psp_spani['V_tot_rtn'])**2).to(u.nPa)

In [ ]:
qt_psp_spc

In [ ]:
qt_psp_spani

In [ ]:
df_psp_spc = qt_psp_spc.to_pandas(index='index')
df_psp_spc.index.name = None

In [ ]:
df_psp_spani = qt_psp_spani.to_pandas(index='index')
df_psp_spani.index.name = None

Some SPAN-i temperature values are negative. Check this and clean the data afterwards

In [ ]:
try:
    print(np.nanmin(df_psp_spani['Temperature']))
    print(np.nanmin(df_psp_spani['T_K']))
    df_psp_spani['T_K'].plot()
    df_psp_spani['Temperature'].plot()
    plt.ylim(-18.002, 0.0)
    plt.show()
except:
    print('not possible. no data?')

#### clean data

In [ ]:
if plot_epilo_e:
    electron_countrate_keys = df_psp_epilo_org.filter(like='Electron_CountRate_ChanF_E').keys()
    df_psp_epilo_org[electron_countrate_keys] = df_psp_epilo_org[electron_countrate_keys].mask(df_psp_epilo_org[electron_countrate_keys] < 0.0)

In [ ]:
df_psp_spc = df_psp_spc.mask(df_psp_spc['general_flag']!=0.0)

In [ ]:
df_psp_spani['Temperature'].mask(df_psp_spani['Temperature']<0.0, inplace=True)
df_psp_spani['T_K'].mask(df_psp_spani['T_K']<0.0, inplace=True)

In [ ]:
# gf_mask = qt_psp_spc['general_flag']!=0.0
# for key in qt_psp_spc.columns.keys():
#     if key != 'index':
#         qt_psp_spc[key].mask = gf_mask

In [ ]:
# SPAN-I
# df_psp_spani_vp_rtn_sun['V_tot_rtn'] = np.sqrt(df_psp_spani_vp_rtn_sun['Vx RTN']**2+df_psp_spani_vp_rtn_sun['Vy RTN']**2+df_psp_spani_vp_rtn_sun['Vz RTN']**2)

# df_psp_spani_T['T_K'] = df_psp_spani_T['Temperature']/k_B.value*e.value
# df_psp_spani_T['T_1000K'] = df_psp_spani_T['Temperature']/k_B.value*e.value/1000

# df_psp_spani = pd.concat([df_psp_spani_np, df_psp_spani_vp_rtn_sun, df_psp_spani_T, df_psp_spani_QF], axis=1)

#### Filter data based on Quality Flags
The Quality flags mostly contain a description of the instrument activities and operational status. For those, I would recommend avoiding anything with the following quality bits set to 1:

- bit0 - counter overflow
- bit3 - spoiler test
- bit10 - bad energy table
- bit11 - MCP test
- bit14 - threshold test
- bit15 - commanding

(R. Livi, priv. comm.)

In [ ]:
df_psp_spani['Quality Flag binary'] = df_psp_spani['Quality Flag'].astype(int).map('{:b}'.format).astype(str)
df_psp_spani['Quality Flag binary'] = df_psp_spani['Quality Flag binary'].str.zfill(16)

qf_bits_list = ['Counter Overflow', 'Survey Snapshot ON (not applicable to archive products)', 'Alternate Energy Table', 'Spoiler Test', 'Attenuator Engaged', 'Highest Archive Rate', 'No Targeted Sweep',
                'SPAN-Ion New Mass Table (not applicable to electrons)', 'Over-deflection', 'Archive Snapshot ON', 'Bad Energy Table', 'MCP Test', 'Survey Available', 'Archive Available', 
                'Threshold Test', 'Commanding']
qf_bits_list.reverse()

for i in range(len(qf_bits_list)):
    df_psp_spani[qf_bits_list[i]] = df_psp_spani['Quality Flag binary'].str[i]
    df_psp_spani[qf_bits_list[i]] = df_psp_spani[qf_bits_list[i]].astype(int)

cond1 = df_psp_spani['Counter Overflow']==1
cond2 = df_psp_spani['Spoiler Test']==1
cond3 = df_psp_spani['Bad Energy Table']==1
cond4 = df_psp_spani['MCP Test']==1
cond5 = df_psp_spani['Threshold Test']==1
cond6 = df_psp_spani['Commanding']==1

df_psp_spani = df_psp_spani.mask(cond1 | cond2 | cond3 | cond4 | cond5 | cond6)

Drop binary version of Quality Flag because otherwise resampling will crash later:

In [ ]:
df_psp_spani.drop(columns='Quality Flag binary', inplace=True)

## Resampling of loaded data

In [ ]:
if resample is not None:
    if plot_epihi_e or plot_epihi_p:
        df_psp_het = resample_df(df_psp_het_org, resample)   
    if plot_epilo_e:
        df_psp_epilo = resample_df(df_psp_epilo_org, resample) 
    if plot_epilo_p:
        df_psp_epilo_ic = resample_df(df_psp_epilo_ic_org, resample) 
    # if plot_psp_pixel:
    #     df_psp_pixel = resample_df(df_psp_pixel_org, resample) 
    if plot_Vsw or plot_N or plot_T:
        df_magplas_spani = resample_df(df_psp_spani, resample_mag) 
        df_magplas_spc = resample_df(df_psp_spc, resample_mag)
    if plot_mag:
        df_mag = resample_df(df_psp_mag, resample_mag) 
    if plot_stix:
        df_stix = resample_df(df_stix_orig, resample) 
 
else:
    if plot_epihi_e or plot_epihi_p:
        df_psp_het = df_psp_het_org 
    if plot_epilo_e:
        df_psp_epilo = df_psp_epilo_org
    if plot_epilo_p:
        df_psp_epilo_ic = df_psp_epilo_ic_org
    # if plot_psp_pixel:
    #     df_psp_pixel = df_psp_pixel_org
    if plot_Vsw or plot_N or plot_T:
        df_magplas_spani = df_psp_spani 
        df_magplas_spc = df_psp_spc
    if plot_mag:
        df_mag = df_psp_mag
    if plot_stix:
        df_stix = df_stix_orig 

## Energy channel selection

In [ ]:
#Choose every n:th channel
n_psp_het_e = 2
n_psp_het_p = 1
n_psp_epilo_e = 1
n_psp_epilo_ic = 4

#Channels list
channels_n_psp_het_e = list(np.arange(0, len(df_psp_het.filter(like=f'{psp_het_viewing}_Electrons_Rate_').keys()), n_psp_het_e))
channels_n_psp_het_p = list(np.arange(0, len(df_psp_het.filter(like=f'{psp_het_viewing}_H_Flux_').keys()), n_psp_het_p))
channels_n_psp_epilo_e = list(np.arange(3, 8, n_psp_epilo_e)) # list(np.arange(0, len(df_psp_epilo.filter(like='Electron_CountRate_ChanF').keys()), n_psp_epilo_e))
channels_n_psp_epilo_ic = list(np.arange(0, 31, n_psp_epilo_ic))

# channels_n_psp_epilo_e = [ 10, 11]
# channels_n_psp_epilo_ic = []
# channels_n_psp_het_e = [0, 1, 2]

#Chosen channels
print('Channels:')
print('psp_het_e:', channels_n_psp_het_e, ',', len(channels_n_psp_het_e))
print('psp_het_p:', channels_n_psp_het_p, ',', len(channels_n_psp_het_p))
print('psp_epilo_e:', channels_n_psp_epilo_e, ',', len(channels_n_psp_epilo_e))
print('psp_epilo_ic:', channels_n_psp_epilo_ic, ',', len(channels_n_psp_epilo_ic))

In [ ]:
print(f'HET {psp_het_viewing}'+psp_het_energies['H_ENERGY_LABL'][0][0])
print(df_psp_het[f'{psp_het_viewing}_H_Flux_0'].sum())

In [ ]:
def make_plot():
    # %matplotlib notebook
    %matplotlib inline
    # %matplotlib ipympl 
    # %matplotlib widget
    
    font_ylabel = 20
    font_legend = 10
    
    if panels == 3:
        fig, axs = plt.subplots(nrows=panels, sharex=True, figsize=[12, 4*panels])#, gridspec_kw={'height_ratios': panel_ratios})# layout="constrained")
    else:
        fig, axs = plt.subplots(nrows=panels, sharex=True, figsize=[12, 3*panels], gridspec_kw={'height_ratios': panel_ratios})# layout="constrained")
    fig.subplots_adjust(hspace=0.1)
    
    i = 0
    
    if plot_stix:
        for key in df_stix.keys():
            axs[i].plot(df_stix.index, df_stix[key], ds="steps-mid", label=key)
        if stix_ltc:
            title = 'SolO/STIX (light travel time corrected)'
        else:
            title = 'SolO/STIX'
        if legends_inside:
            axs[i].legend(loc='upper right', title=title)
        else:
            # axs[i].legend(loc='upper right', title=title, bbox_to_anchor=(1, 0.5))
            axs[i].legend(bbox_to_anchor=(1.01, 1), loc='upper left', title=title)
        axs[i].set_ylabel('Counts', fontsize=font_ylabel)
        axs[i].set_yscale('log')
        i +=1  
    
    
    color_offset = 4 
    
    if plot_electrons:
        if plot_epilo_e:
            axs[i].set_prop_cycle('color', plt.cm.viridis_r(np.linspace(0, 1, len(channels_n_psp_epilo_e)+color_offset)))
            for channel in channels_n_psp_epilo_e:
                psp_epilo_energy = np.round(psp_epilo_energies_org[f'Electron_Chan{epilo_channel}_Energy'][f'Electron_Chan{epilo_channel}_Energy_E{channel}_P{epilo_viewing}'], 2).astype(str)
                axs[i].plot(df_psp_epilo.index, df_psp_epilo[f'Electron_CountRate_Chan{epilo_channel}_E{channel}_P{epilo_viewing}'],
                            ds="steps-mid", label=f'EPI-lo PE {epilo_channel}{epilo_viewing} {psp_epilo_energy} keV')
    
        if plot_epihi_e:
            axs[i].set_prop_cycle('color', plt.cm.Reds_r(np.linspace(0, 1, len(channels_n_psp_het_e)+color_offset)))
            for channel in channels_n_psp_het_e:
                axs[i].plot(df_psp_het.index, df_psp_het[f'{psp_het_viewing}_Electrons_Rate_{channel}'],
                            ds="steps-mid", label=f'HET {psp_het_viewing}'+psp_het_energies['Electrons_ENERGY_LABL'][channel][0])
                
        # axs[i].set_ylabel("Flux\n"+r"[(cm$^2$ sr s MeV)$^{-1}]$", fontsize=font_ylabel)
        axs[i].set_ylabel("Count rates", fontsize=font_ylabel)
        if legends_inside:
            axs[i].legend(loc='upper right', borderaxespad=0., 
                          title=f'Electrons',
                          fontsize=font_legend)
        else:
            axs[i].legend(bbox_to_anchor=(1.01, 1), loc='upper left', borderaxespad=0., 
                          title=f'Electrons',
                          fontsize=font_legend)
       
        axs[i].set_yscale('log')
        i +=1    
    
        
    color_offset = 2    
    if plot_protons:
        if plot_epilo_p:
            axs[i].set_prop_cycle('color', plt.cm.viridis_r(np.linspace(0, 1, len(channels_n_psp_epilo_ic)+color_offset)))
            # [::-1] to reverse list
            for channel in channels_n_psp_epilo_ic[::-1]:
                # print(f'H_Flux_Chan{epilo_ic_channel}_E{channel}_P{epilo_ic_viewing}')
                psp_epilo_ic_energy = np.round(psp_epilo_ic_energies_org[f'H_Chan{epilo_ic_channel}_Energy'][f'H_Chan{epilo_ic_channel}_Energy_E{channel}_P{epilo_ic_viewing}'], 2).astype(str)
                axs[i].plot(df_psp_epilo_ic.index, df_psp_epilo_ic[f'H_Flux_Chan{epilo_ic_channel}_E{channel}_P{epilo_ic_viewing}'],
                            ds="steps-mid", label=f'EPI-lo IC {epilo_ic_channel}{epilo_ic_viewing} {psp_epilo_ic_energy} keV')
    
        # if plot_psp_pixel:
        #     axs[i].set_prop_cycle('color', plt.cm.tab10(range(6)))
        #     for key in ['L2Ap', 'L3Ap', 'L4Ap', 'H2Ap', 'H3Ap', 'H4Ap']:
        #     # for key in ['L2Ap', 'L4Ap', 'H2Ap', 'H3Ap', 'H4Ap']:
        #         axs[i].plot(df_psp_pixel.index, df_psp_pixel[f'{key}_Flux'], label=f'{key} {energies_psp_pixel[key]}', drawstyle='steps-mid')
        
        if plot_epihi_p:    
            if plot_epihi_p_combined_pixels:
                # comb_channels = [[1,2], [3,5], [5,7], [4,5], [7], [9]]
                comb_channels = [[3,5], [5,7], [4,5], [7], [9]]
                axs[i].set_prop_cycle('color', plt.cm.Greys_r(np.linspace(0, 1, len(comb_channels)+5)))
                for channel in comb_channels:
                    df_psp_epihi, df_psp_epihi_name = calc_av_en_flux_PSP_EPIHI(df_psp_het, psp_het_energies, channel, 'p', 'het', psp_het_viewing)
                    axs[i].plot(df_psp_epihi.index, df_psp_epihi.flux, label=f'HET {psp_het_viewing} {df_psp_epihi_name}', lw=1, ds="steps-mid")
            else:
                axs[i].set_prop_cycle('color', plt.cm.plasma(np.linspace(0, 1, len(channels_n_psp_het_p)+color_offset)))
                for channel in channels_n_psp_het_p:
                    axs[i].plot(df_psp_het.index, df_psp_het[f'{psp_het_viewing}_H_Flux_{channel}'], label=f'HET {psp_het_viewing}'+psp_het_energies['H_ENERGY_LABL'][channel][0], ds="steps-mid")
        
        axs[i].set_ylabel("Flux\n"+r"[(cm$^2$ sr s MeV)$^{-1}]$", fontsize=font_ylabel)
        # title = f'Ions (HET {psp_het_viewing})'
        title = f'Ions (Pixel)'
        if legends_inside:
            axs[i].legend(loc='upper right', borderaxespad=0., 
                          title=title,
                          fontsize=font_legend)
        else:
            axs[i].legend(bbox_to_anchor=(1.01, 1), loc='upper left', borderaxespad=0., 
                          title=title,
                          fontsize=font_legend)
        axs[i].set_yscale('log')
    
        # axs[i].set_ylim([5e-2, None])
        
        i +=1    
        
        
    # plot magnetic field
    if plot_mag:
        ax = axs[i]
        ax.plot(df_mag.index, df_mag['|b|'], label='B', color='k', linewidth=1)
        ax.plot(df_mag.index.values, df_mag['br'].values, label='Br', color='dodgerblue')
        ax.plot(df_mag.index.values, df_mag['bt'].values, label='Bt', color='limegreen')
        ax.plot(df_mag.index.values, df_mag['bn'].values, label='Bn', color='deeppink')
        ax.axhline(y=0, color='gray', linewidth=0.8, linestyle='--')
        if legends_inside:
            ax.legend(loc='upper right')
        else:
            # ax.legend(loc='upper right', bbox_to_anchor=(1.01, 0.5))
            ax.legend(bbox_to_anchor=(1.01, 1), loc='upper left')
            
        ax.set_ylabel('B [nT]', fontsize=font_ylabel)
        ax.tick_params(axis="x", direction="in", which='both')#, pad=-15)
        i += 1
        
    if plot_polarity:
        pos = get_horizons_coord(f'PSP', time={'start':df_mag.index[0]-pd.Timedelta(minutes=15), 'stop':df_mag.index[-1]+pd.Timedelta(minutes=15), 'step':"1min"})  # (lon, lat, radius) in (deg, deg, AU)
        pos = pos.transform_to(frames.HeliographicStonyhurst())
        #Interpolate position data to magnetic field data cadence
        r = np.interp([t.timestamp() for t in df_mag.index], [t.timestamp() for t in pd.to_datetime(pos.obstime.value)], pos.radius.value)
        lat = np.interp([t.timestamp() for t in df_mag.index], [t.timestamp() for t in pd.to_datetime(pos.obstime.value)], pos.lat.value)
        pol, phi_relative = polarity_rtn(df_mag['br'].values, df_mag['bt'].values, df_mag['bn'].values, r, lat, V=400)
        # create an inset axe in the current axe:
        pol_ax = inset_axes(ax, height="5%", width="100%", loc='upper center', bbox_to_anchor=(0.,0,1,1.1), bbox_transform=ax.transAxes) # center, you can check the different codes in plt.legend?
        pol_ax.get_xaxis().set_visible(False)
        pol_ax.get_yaxis().set_visible(False)
        pol_ax.set_ylim(0,1)
        pol_ax.set_xlim([df_mag.index.values[0], df_mag.index.values[-1]])
        pol_arr = np.zeros(len(pol))+1
        timestamp = df_mag.index.values[2] - df_mag.index.values[1]
        norm = Normalize(vmin=0, vmax=180, clip=True)
        mapper = cm.ScalarMappable(norm=norm, cmap=cm.bwr)
        pol_ax.bar(df_mag.index.values[(phi_relative>=0) & (phi_relative<180)],pol_arr[(phi_relative>=0) & (phi_relative<180)],color=mapper.to_rgba(phi_relative[(phi_relative>=0) & (phi_relative<180)]),width=timestamp)
        pol_ax.bar(df_mag.index.values[(phi_relative>=180) & (phi_relative<360)],pol_arr[(phi_relative>=180) & (phi_relative<360)],color=mapper.to_rgba(np.abs(360-phi_relative[(phi_relative>=180) & (phi_relative<360)])),width=timestamp)
        pol_ax.set_xlim(startdate, enddate)
        
    if plot_mag_angles:
        ax = axs[i]
        #Bmag = np.sqrt(np.nansum((mag_data.B_r.values**2,mag_data.B_t.values**2,mag_data.B_n.values**2), axis=0))    
        # alpha, phi = mag_angles(df_mag.BFIELD_3, df_mag.BFIELD_0.values, df_mag.BFIELD_1.values,
        #                         df_mag.BFIELD_2.values)
        ax.plot(df_mag.index, df_mag['theta'], '.k', label='theta', ms=1)
        ax.axhline(y=0, color='gray', linewidth=0.8, linestyle='--')
        ax.set_ylim(-90, 90)
        ax.set_ylabel(r"$\Theta_\mathrm{B}$ [°]", fontsize=font_ylabel)
        ax.tick_params(axis="x",direction="in", pad=-15)
    
        i += 1
        ax = axs[i]
        # ax.plot(df_mag.index, df_mag['phi'], '.k', label='phi', ms=1)
        ax.plot(df_mag.index, df_mag['phi_mod'], '.k', label='phi', ms=1)
        # ax.plot(df_mag.index, df_mag['phi2'], '.r', label='phi', ms=1)    
        ax.axhline(y=0, color='gray', linewidth=0.8, linestyle='--')
        ax.set_ylim(-180, 180)
        ax.set_ylabel(r"$\Phi_\mathrm{B}$ [°]", fontsize=font_ylabel)
        ax.tick_params(axis="x",direction="in", which='both', pad=-15)
        i += 1
        
    ### Temperature
    if plot_T:
        axs[i].plot(df_magplas_spani.index, df_magplas_spani['T_K'], '-k', label="SPAN-i")
        axs[i].plot(df_magplas_spc.index, df_magplas_spc['T'], '-r', label="SPC")
        axs[i].set_ylabel(r"T$_\mathrm{p}$ [K]", fontsize=font_ylabel)
        axs[i].set_yscale('log')
    
        # TODO: manually set lower boundary, remove at some point
        axs[i].set_ylim(np.nanmin(df_magplas_spc['T'])-0.1*np.nanmin(df_magplas_spc['T']), None)
    
        if legends_inside:
            axs[i].legend(loc='upper right')
        else:
            axs[i].legend(bbox_to_anchor=(1.01, 1), loc='upper left')
        i += 1
    
    ### Dynamic pressure
    if plot_p_dyn:
        axs[i].plot(df_magplas_spani.index, df_magplas_spani['p_dyn'], '-k', label="SPAN-i")
        axs[i].plot(df_magplas_spc.index, df_magplas_spc['p_dyn'], '-r', label="SPC")
        axs[i].set_ylabel(r"P$_\mathrm{dyn}$ [nPa]", fontsize=font_ylabel)
        if legends_inside:
            axs[i].legend(loc='upper right')
        else:
            axs[i].legend(bbox_to_anchor=(1.01, 1), loc='upper left')
        axs[i].set_yscale('log')
        i += 1
    
    ### Density
    if plot_N:
        axs[i].plot(df_magplas_spani.index, df_magplas_spani['Density'], '-k', label="SPAN-i")
        axs[i].plot(df_magplas_spc.index, df_magplas_spc['np_tot'], '-r', label="SPC")
        axs[i].set_ylabel(r"N$_\mathrm{p}$ [cm$^{-3}$]", fontsize=font_ylabel)
        if legends_inside:
            axs[i].legend(loc='upper right')
        else:
            axs[i].legend(bbox_to_anchor=(1.01, 1), loc='upper left')
        axs[i].set_yscale('log')
        i += 1
    
    ### Vsw
    if plot_Vsw:
        axs[i].plot(df_magplas_spani.index, df_magplas_spani['V_tot_rtn'], '-k', label="SPAN-i")
        axs[i].plot(df_magplas_spc.index, df_magplas_spc['|vp_tot|'], '-r', label="SPC")
        axs[i].set_ylabel(r"V$_\mathrm{sw}$ [kms$^{-1}$]", fontsize=font_ylabel)
        if legends_inside:
            axs[i].legend(loc='upper right')
        else:
            axs[i].legend(bbox_to_anchor=(1.01, 1), loc='upper left')
        # i += 1     
            
    #axs[-1].set_xlabel(f"Date in {year}/  Time (UTC)", fontsize=15)
    #axs[-1].set_xlim(startdate, enddate)
    axs[0].set_title(f'Parker Solar Probe', ha='center')
    axs[-1].xaxis.set_major_formatter(mdates.DateFormatter('%H:%M\n%m-%d'))
    axs[-1].xaxis.set_tick_params(rotation=0)
    axs[-1].set_xlabel(f"Time (UTC) / Date in {year}")#, fontsize=15)
    axs[-1].set_xlim(startdate, enddate)
    
    #plt.tight_layout()
    fig.patch.set_facecolor('white')
    fig.set_dpi(200)
    return

In [ ]:
make_plot()

In [ ]:
# fig.savefig('figure.png', dpi=200)

## EOF